# Description of Notebook
In this Notebook we will create the states for the Job Scheduling Problem in the deterministic case.<br>
In every non-final state an action can be taken, leading to one of the successor states. Therefore, we construct a function that computes every possible state of a given problem environment by representing it with an initial state and iteratively transitioning to every feasible successor state.<br> Here, a state corresponds to a moment in a schedule in which a decision has to be taken, i.e. a Machine becomes free. The actions therefore consist of turning it off or assigning a Job to it. Since we approach this problem from the perspective of Deep Q-Learning, every state needs a Q-value for every feasible action. These will induce the target values. Finally, we will define a composite function executing all these steps.<br>

Policies can be deduced from the states and their Q-values. These are equivalent to scheduling rules. A function will be given to compute the optimal as well as a random Policy.

# Code

In [ ]:
#import dependencies
import operator
import random
import numpy as np
import copy
import time
import pickle
import import_ipynb
from Jobs_and_Machines import *
from Global_Variables import *

### Class of States
The idea is that the Machines will be processing the Jobs. Whenever a Job finishes, a Machine gets free and therefore an action has to be taken. This situation will be represented with a State.<br>
Feasible actions in this state are to assign a pending Job to the free Machine or to shut it down, which means it cannot be used anymore from here on.<br>
If all Jobs have been assigned and/or processed already, the state is final and no decision has to be taken.<br>
In case that more than one Machine is free at some point, we will always use the free Machine with the lowest index first. After having taken an action on it, it is not free anymore and we move to the next state, which will happen to have the same time, to take care of the next free Machine.

A State consists of these environmental information:

   1. Time
   2. Remaining Jobs
   3. Machines that are still on duty
   4. Machine upon which an action shall be performed
   5. Free Machines
   6. Remaining processing time of occupated Machines
    
The State also needs information about how it is related to other States:
   1. ID of State
   2. Predecessor State
   3. Action that led to this state
   4. Transition Costs from predecessor to current State
   5. A dictionary mapping every action to the corresponding succesor states as well as a list of all successor States
   6. Optimal Future Costs for every action (Q-Values)
   
Finally, it needs its information stored as data that the Neural Network can process:
   1. Input
   2. Target
   
However, these will only be computed in the Notebook [Data_for_NN.ipynb](https://github.com/Dieguinho1612/Job-Scheduling-Deep-Reinforcement-Learning/blob/main/Notebooks/Data_for_NN.ipynb).

In [ ]:
class states:
    def __init__(self, time, jobs_remaining, machines_on_duty,
                 free_machines, machine_runtimes, predecessor):
        
        #time
        self.time = time
        
        #information about the jobs
        self.jobs_remaining = jobs_remaining #"one" if job is still remaining, "zero" if it was assigned already
        
        #information about the machines
        self.machines_on_duty = machines_on_duty #"one" if machine is still on duty, "zero" if it was shut down already
        self.machine = None #index of the free machine on which an action shall be taken, will be added later
        self.free_machines = free_machines #"one" if machine is free, "zero" if it is occupied or shut down
        self.machine_runtimes = machine_runtimes #remaining runtime of every machine
        
        #predecessor and successors
        self.ID = None #ID of the state, will be given after all states were created
        self.predecessor = predecessor #what was the predecessor state
        self.successors = None #the successor states will be added when creating the entire tree of states
        
        #costs and actions
        self.costs = None #cost to transition from the predecessor state to the current one
        self.action = (None, None) #action to transition from the predecessor state to the current one
        self.transition_dic = {} #dictionary of all feasible actions and the successor states to which they lead
        
        #optimal future costs
        self.backtracking = 0 #will be used in calculating the Qvalues
        self.Qvalues = [None]*(n+1) #i-th entry stands for assigning job i, n+1-entry for deactivating currently free machine
        
        #data for Neural Network
        self.input = None
        self.target = None

### Creation of States
We will now define all necessary functions so that the states can be created.

In [ ]:
#Create every possible state
def create_all_states(from_state=None):
    
    #initiate list of states
    list_states = []
    
    #if not starting from a current state, create initial state
    if from_state:
        initial_state = from_state
    else:
        initial_state = create_initial_state()
    ID = 0
    
    #list of current states
    current_states = [initial_state]
    
    #go through every current state, save their successors, add current states to list of all states
        #then define successor states as current states, clear list of successor states and repeat until done
    while current_states:
        
        #empty list of all successor states
        successor_states = []
        
        #create and add all successor states of current states
        for state in current_states:
            
            #give ID
            state.ID = ID
            ID += 1
            
            #list of all successors of this state
            state_successors = []
            
            #create one state for every remaining job assigned to the free machine with lowest index
            if sum(state.jobs_remaining) > 0:
                
                #get current machine as object
                machine = list_machines[state.machine]
                
                #loop through jobs
                for index, job in enumerate(list_jobs):
                    #check if the job still has to be done
                    if state.jobs_remaining[index] == 1:
                        #assign job to machine
                        state_successors.append(assign_job(state,job,machine))    
                
                #check if turning it off is an option
                if sum(state.machines_on_duty) > 1:                       
                    #add successor state created by shutting down machine
                    state_successors.append(turn_off_machine(state, machine))
                        
            
            #add successor list to the attributes of state
            state.successors = state_successors
            
            #add successors of this state to the list of all successors of all current states
            successor_states += state_successors
        
        #add current states to list of all states
        list_states += current_states
        
        #the successor states then become the current states
        current_states = successor_states
        
    return list_states

In [ ]:
#create the initial state
def create_initial_state():
    
    #create initial circumstances
    time = 0
    jobs_remaining = [1]*n #every job remains
    machines_on_duty = [1]*m #every machine is on duty
    machine_runtimes = [machine.init_runtime for machine in list_machines] #machine runtimes equal their initial occupations
    free_machines = [1 if x==0 else 0 for x in machine_runtimes] #start in first free machine
    
    #create initial state
    initial_state = states(time, jobs_remaining, machines_on_duty, free_machines, machine_runtimes, None)
    initial_state.costs = initialize_costs(initial_state)
    initial_state.machine = free_machines.index(1)
    
    return initial_state

In [ ]:
#transition to successor state by assigning a job
def assign_job(state,job,machine):
    
    #job gets canceled from to-do-list
    jobs_remaining = state.jobs_remaining.copy()
    jobs_remaining[job.index] = 0
    
    #machine is not free anymore
    free_machines = state.free_machines.copy()
    free_machines[machine.index] = 0
    
    #it gets a runtime equivalent to the jobs processing time
    machine_runtimes = state.machine_runtimes.copy()
    machine_runtimes[machine.index] = job.processing_time [machine.index]
    
    #if there is another free machine, we create a successor state at same time
    if sum(free_machines) > 0:
        #no time passes
        time_difference = 0
        #create state
        successor_state = states(state.time, jobs_remaining, state.machines_on_duty,
                                 free_machines, machine_runtimes, state)
    
    #elsewise we have to proceed to the point where the next job finishes
    else:
        #calculate new time for when next job finishes
        time_difference = min([runtime for runtime in machine_runtimes if runtime > 0])
        new_time = state.time + time_difference
                            
        #the following machine(s) become free
        for index in range(m):
            if machine_runtimes[index] == time_difference:
                free_machines[index] = 1
                            
        #update machine runtimes
        machine_runtimes = [max(runtime - time_difference,0) for runtime in machine_runtimes]
                            
        #create succesor state
        successor_state = states(new_time, jobs_remaining, state.machines_on_duty,
                                 free_machines, machine_runtimes, state)
        
    #give its action and costs
    successor_state.action = (job.index,machine.index)
    successor_state.costs = transition_costs(state,successor_state,job,machine)
    
    #give free machine for successor state
    successor_state.machine = free_machines.index(1)
        
    #add successor state to transition dictionary
    state.transition_dic[(job.index,machine.index)] = successor_state    
    
    return successor_state

In [ ]:
#transition to successor state of deactivating machine
def turn_off_machine(state, machine):
    
    #machine gets shut down
    machines_on_duty = state.machines_on_duty.copy()
    machines_on_duty[machine.index] = 0
                        
    #machine is not free anymore
    free_machines = state.free_machines.copy()
    free_machines[machine.index] = 0                              
                        
    #if there is another free machine, we create a successor state at same time
    if sum(free_machines) > 0:
        #no time passes
        time_difference = 0
        #create state
        successor_state = states(state.time, state.jobs_remaining, machines_on_duty,
                                 free_machines, state.machine_runtimes, state)
        
    #elsewise we have to proceed to the point where the next job finishes
    else:
        #calculate new time for when next job finishes
        time_difference = min([runtime for runtime in state.machine_runtimes if runtime > 0])
        new_time = state.time + time_difference
                            
        #the following machine(s) become free
        for index in range(m):
            if state.machine_runtimes[index] == time_difference:
                free_machines[index] = 1
                            
        #update machine runtimes
        machine_runtimes = [max(runtime - time_difference,0) for runtime in state.machine_runtimes]
                            
        #create succesor state
        successor_state = states(new_time, state.jobs_remaining, machines_on_duty,
                                 free_machines, machine_runtimes, state)
        
    #give its action and costs
    successor_state.action = (n,machine.index)
    successor_state.costs = transition_costs(state,successor_state,None,machine)
        
    #give free machine for successor state
    successor_state.machine = free_machines.index(1)
    
    #add successor state to transition dictionary
    state.transition_dic[(n,machine.index)] = successor_state
        
    return successor_state

In [ ]:
#initial costs may exist due to initial machine occupations exceeding machine deadlines
def initialize_costs(initial_state):
    
    costs = 0
    for i, runtime in enumerate(initial_state.machine_runtimes):
        machine = list_machines[i]
        #check if initial machine runtime is longer than its deadline
        costs += max(0,machine.weight*(runtime - machine.deadline))
        
    return costs

In [ ]:
#costs of transitioning from one state to one of its successors
def transition_costs(state, successor_state, job, machine):
    
    #runtime costs (time passing + deadline cost of idle jobs)
    st = state.time #start time
    nt = successor_state.time #next time
    transition_costs = nt - st #cost of time passing
    #jobs can be idle, therefore deadline costs might have to be payed while waiting for next state
    for i, job_i in enumerate(list_jobs):
        if successor_state.jobs_remaining[i] == 1: #check if job has been assigned already
            transition_costs += max(0, job_i.weight*(nt-max(job_i.deadline,st)))   
    
    #deadline costs (if job was assigned)
    if job:
        #time for the machine to process the assigned job
        proc_time = job.processing_time[machine.index]
        #completion time
        ct = st + proc_time
        #all deadline costs of assigned job until its done
        transition_costs += max(0,job.weight*(ct - max(job.deadline,st)))
        #all deadline costs of machine until it finishes the assigned job
        transition_costs += max(0,machine.weight*(ct - max(machine.deadline,st)))
        
    #check if state is final
    if sum(successor_state.jobs_remaining) == 0:
        #add remaining runtime costs until last job finishes
        if successor_state.machine_runtimes:
            transition_costs += max(successor_state.machine_runtimes)

    return transition_costs

### Q-Values
We will now define the functions that compute the Q-Values for every feasible action of every State.<br>
The problem structure is a directed tree, so we start with the final-states being the leaves and iteratively backtrack to the respective predecessors.

In [ ]:
#function to get the minimum Qvalue and its belonging action from a state
def minimum(state):
    minimum = None
    job = None

    for i, val in enumerate(state.Qvalues):
        #non-feasible actions would have None as Q-value since no successor state exists for them
        if val != None:
            if minimum == None:
                minimum = val
                job = i
            elif val <= minimum: #we use ""<="" so that there is an emphasis on also considering the jobs with higher index
                minimum = val
                job = i
    
    #final states have no successors, so we set all future costs to zero
    if minimum == None:
        minimum = 0
    
    action = (job,state.machine)
        
    return minimum, action

In [ ]:
#find Q-values of every state by backtracking
def backtracking(all_states):
    
    #list of all states that have already been backtracked completely
    backtracked_states = []
    #list of states that yet have to be backtracked
    states_to_backtrack = all_states.copy()
    
    for state in states_to_backtrack:
        #how many successors of this node have already been backtracked
        state.backtracking = - len(state.successors) #count upwards until zero
                
    while states_to_backtrack:
        #states that are temporary final in regard to backtracking (all their successors have been backtracked already)
        temp_final_states = []
        
        #sort list, so we can pop all nodes that are temporary final in every step
        states_to_backtrack.sort(key=operator.attrgetter('backtracking'))
        for state in states_to_backtrack[::-1]:
            if state.backtracking == 0:
                temp_final_states.append(states_to_backtrack.pop())
            else:
                break
        
        #add values of temporary final states to the optimal future costs of their predecessors at its place for this action
        for state in temp_final_states:
            predecessor = state.predecessor
            #stop when there is no predecessor anymore
            if not predecessor:
                break
            #add optimal future costs at position of action
            predecessor.Qvalues[state.action[0]] = state.costs + minimum(state)[0]
            #count upwards of how many successors this state still needs its Q-value before becoming temporary final itself
            predecessor.backtracking += 1    

### Computation of all states
We now merge all of our former results into one single function that shall compute all possible states together with all of their Q-Values.

In [ ]:
#the entire process from creation of all states to updating their Qvalues by backtracking
def compute_all_states(jobs, machines, from_state=None, MVS=0, JS=0, save=False, name="all_states"):
    
    #define global parameters for easier access
    global list_jobs, list_machines, n,m
    list_jobs, list_machines = jobs, machines
    n, m = len(jobs), len(machines)
    
    #measure starting time
    st = time.time()
    
    #create all states
    all_states = create_all_states(from_state=from_state)
    
    #update their Qvalues by backtracking
    backtracking(all_states)
    
    #if desired, save file of computed states
    if save:
        path = f'{name}.pickle'
        with open(path, 'wb') as f:
            pickle.dump(all_states, f, pickle.HIGHEST_PROTOCOL)
    
    #measure end time
    et = time.time()
    
    #tell how much time the entire process took
    print(round(et-st,2), "seconds to compute", len(all_states), "states.")
    
    return(all_states)

### Policies
Having created all the states and the Q-Values corresponding to each of their feasoble actions, we can now create Policies.<br>
A Policy is a 2-tuple. The first element is the list of all actions, the second one the list of all States.

In [ ]:
#compute random policy
def random_policy(all_states):
    
    #initial state
    rand_state = all_states[0]
    
    random_actions = []
    random_states = [rand_state]
    
    #while state not final
    while rand_state.successors:
        #choose a random successor state
        rand_successor = random.choice(rand_state.successors)
        #add this random action and state to policy
        random_actions.append(rand_successor.action)
        random_states.append(rand_successor)
        #continue policy from this state
        rand_state = rand_successor
        
    return random_actions, random_states

In [ ]:
#compute optimal policy
def optimal_policy(all_states, name="optimal_policy", save=False):
    
    initial_state = all_states[0]
    optimal_actions = []
    optimal_states = []
    state = initial_state
    
    #go down the tree, always choosing the successor with the minimal Q-value
    while state.successors:
        #add state
        optimal_states.append(state)
        #add action
        optimal_action = minimum(state)[1]
        optimal_actions.append(optimal_action)
        #declare the successor
        state = state.transition_dic[optimal_action]
        
    #add final state            
    optimal_states.append(state)
    
    #if desired, safe optimal policy as file
    if save:
        with open(f'{name}.pickle', 'wb') as f:
            pickle.dump((optimal_actions,optimal_states), f, pickle.HIGHEST_PROTOCOL)

    return optimal_actions,optimal_states